In [7]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from torch.utils.data import DataLoader

In [3]:
#Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 46.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 27.1MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 99.9MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.89MB/s]


#Knowledge Distillation
##Teacher Model


In [4]:
class TeacherNet(nn.Module):
    def __init__(self):
        super(TeacherNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(5, 5)
        self.fc1 = nn.Linear(32 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

##Evaluation function


In [5]:

def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

##Initialize and train the teacher model


In [9]:
teacher_model = TeacherNet()
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
teacher_criterion = nn.CrossEntropyLoss()
for epoch in range(5):
    teacher_model.train()
    running_loss = 0.0

    for data in tqdm(trainloader):
        inputs, labels = data
        teacher_optimizer.zero_grad()
        outputs = teacher_model(inputs)
        loss = teacher_criterion(outputs, labels)
        loss.backward()
        teacher_optimizer.step()

        running_loss += loss.item()

    teacher_accuracy = evaluate(teacher_model)

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}, Accuracy: {teacher_accuracy * 100:.2f}%")

100%|██████████| 938/938 [00:38<00:00, 24.27it/s]


Epoch 1, Loss: 0.21660741983748066, Accuracy: 97.57%


100%|██████████| 938/938 [00:36<00:00, 25.47it/s]


Epoch 2, Loss: 0.07360889268011403, Accuracy: 98.15%


100%|██████████| 938/938 [00:37<00:00, 25.00it/s]


Epoch 3, Loss: 0.05601034780255835, Accuracy: 97.93%


100%|██████████| 938/938 [00:37<00:00, 24.93it/s]


Epoch 4, Loss: 0.046822121343029335, Accuracy: 98.89%


100%|██████████| 938/938 [00:38<00:00, 24.67it/s]


Epoch 5, Loss: 0.03901709357439392, Accuracy: 98.23%


##Student Model


In [8]:

class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

##Initialize and train the teacher model


In [10]:

student_model = StudentNet()
student_optimizer = optim.Adam(student_model.parameters(), lr=0.001)

##Loss function (KL Divergence)


In [11]:

def knowledge_distillation_loss(student_logits, teacher_logits):
    p_teacher = F.softmax(teacher_logits , dim=1)
    p_student = F.log_softmax(student_logits, dim=1)
    loss = F.kl_div(p_student, p_teacher, reduction='batchmean')
    return loss

## Train the student model with knowledge distillation


In [12]:

for epoch in range(5):  # You can adjust the number of epochs
    student_model.train()
    running_loss = 0.0

    for data in tqdm(trainloader):
        inputs, labels = data
        student_optimizer.zero_grad()
        student_logits = student_model(inputs)
        teacher_logits = teacher_model(inputs).detach()  # Detach the teacher's output to avoid backpropagation
        loss = knowledge_distillation_loss(student_logits, teacher_logits)
        loss.backward()
        student_optimizer.step()

        running_loss += loss.item()

    student_accuracy = evaluate(student_model)

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(testloader)}, Accuracy: {student_accuracy * 100:.2f}%")

100%|██████████| 938/938 [00:24<00:00, 38.26it/s]


Epoch 1, Loss: 2.0235745541419194, Accuracy: 93.30%


100%|██████████| 938/938 [00:25<00:00, 36.14it/s]


Epoch 2, Loss: 0.9613800629688676, Accuracy: 95.70%


100%|██████████| 938/938 [00:25<00:00, 36.36it/s]


Epoch 3, Loss: 0.6861976202173977, Accuracy: 96.45%


100%|██████████| 938/938 [00:25<00:00, 36.57it/s]


Epoch 4, Loss: 0.547288564199902, Accuracy: 96.06%


100%|██████████| 938/938 [00:24<00:00, 38.41it/s]


Epoch 5, Loss: 0.4697081448605201, Accuracy: 96.60%


In [13]:
%timeit evaluate(teacher_model)

4.03 s ± 617 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit evaluate(student_model) #faster

2.41 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
